In [1]:
import os
import json

# Entidades válidas
valid_labels = {
    'B-JURISPRUDENCIA', 'I-JURISPRUDENCIA',
    'B-LEGISLACAO', 'I-LEGISLACAO',
    'B-LOCAL', 'I-LOCAL',
    'B-ORGANIZACAO', 'I-ORGANIZACAO',
    'B-PESSOA', 'I-PESSOA',
    'B-TEMPO', 'I-TEMPO'
}

# Pasta base onde estão as partições
base_dir = "../Base de Dados/leNER/divisions/iterative/"
output_dir = "../Base de Dados/leNER/json_divisions/"
os.makedirs(output_dir, exist_ok=True)

# Função para converter BIO para JSON estruturado
def convert_bio_file_to_json(txt_path):
    with open(txt_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    samples = []
    tokens, labels = [], []

    for line in lines:
        line = line.strip()
        if not line:
            if tokens:
                samples.append((tokens, labels))
                tokens, labels = [], []
            continue
        parts = line.split()
        if len(parts) == 2:
            token, label = parts
            tokens.append(token)
            labels.append(label)
    
    # última frase
    if tokens:
        samples.append((tokens, labels))

    data_json = []
    for tokens, labels in samples:
        text = " ".join(tokens)
        entities = []
        i = 0
        while i < len(labels):
            if labels[i].startswith("B-"):
                label_type = labels[i][2:]
                entity_tokens = [tokens[i]]
                i += 1
                while i < len(labels) and labels[i].startswith("I-") and labels[i][2:] == label_type:
                    entity_tokens.append(tokens[i])
                    i += 1
                entities.append({
                    "text": " ".join(entity_tokens),
                    "label": label_type
                })
            else:
                i += 1
        data_json.append({
            "text": text,
            "entities": entities
        })
    
    return data_json

# Iterar sobre os 10 arquivos de divisão e salvar em JSON
for i in range(10):
    txt_file = os.path.join(base_dir, f"division_{i}.txt")
    json_file = os.path.join(output_dir, f"division_{i}.json")
    json_data = convert_bio_file_to_json(txt_file)
    with open(json_file, "w", encoding="utf-8") as f:
        json.dump(json_data, f, indent=2, ensure_ascii=False)
    print(f"Convertido: {txt_file} → {json_file}")



Convertido: ../Base de Dados/leNER/divisions/iterative/division_0.txt → ../Base de Dados/leNER/json_divisions/division_0.json
Convertido: ../Base de Dados/leNER/divisions/iterative/division_1.txt → ../Base de Dados/leNER/json_divisions/division_1.json
Convertido: ../Base de Dados/leNER/divisions/iterative/division_2.txt → ../Base de Dados/leNER/json_divisions/division_2.json
Convertido: ../Base de Dados/leNER/divisions/iterative/division_3.txt → ../Base de Dados/leNER/json_divisions/division_3.json
Convertido: ../Base de Dados/leNER/divisions/iterative/division_4.txt → ../Base de Dados/leNER/json_divisions/division_4.json
Convertido: ../Base de Dados/leNER/divisions/iterative/division_5.txt → ../Base de Dados/leNER/json_divisions/division_5.json
Convertido: ../Base de Dados/leNER/divisions/iterative/division_6.txt → ../Base de Dados/leNER/json_divisions/division_6.json
Convertido: ../Base de Dados/leNER/divisions/iterative/division_7.txt → ../Base de Dados/leNER/json_divisions/division

In [ ]:
from llama_cpp import Llama
import os
import json
import random
from tqdm import tqdm
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer
import time
import torch  # Para verificar GPU

# Verifica se a GPU está disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")
if torch.cuda.is_available():
    print(f"GPU disponível: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("Nenhuma GPU disponível, utilizando CPU.")

llama_model_path = "/shared/ModelsGGUF/hugging-quants/DeepSeek-R1-Distill-Qwen-14B-GGUF/DeepSeek-R1-Distill-Qwen-14B-Q4_K_M.gguf"

# Inicializa o modelo Llama com GPU
model = Llama(
    model_path=llama_model_path,
    n_ctx=2048,
    n_gpu_layers=30
)

base_dir = "../Base de Dados/leNER/json_divisions/"
division_files = [f"division_{i}.json" for i in range(10)]

def load_data(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def get_all_labels():
    return ["JURISPRUDENCIA", "LEGISLACAO", "LOCAL", "ORGANIZACAO", "PESSOA", "TEMPO"]

def select_minimal_few_shot(dataset):
    all_labels = set(get_all_labels())
    selected = []
    used_ids = set()
    label_to_items = {label: [] for label in all_labels}
    for item in dataset:
        for ent in item['entities']:
            label_to_items[ent['label']].append(item)
    for label in all_labels:
        for item in label_to_items[label]:
            if id(item) not in used_ids:
                selected.append(item)
                used_ids.add(id(item))
                break
    return selected

def truncate_text(text, max_tokens=150):
    return text[:max_tokens] + "..." if len(text) > max_tokens else text

def build_prompt(few_shot_examples, test_text):
    prompt = """\
Tarefa: Extraia entidades nomeadas do texto abaixo e retorne apenas uma lista JSON válida.
Cada entidade deve ter os campos: "text" e "label".

As entidades possíveis são:
- JURISPRUDENCIA
- LEGISLACAO
- LOCAL
- ORGANIZACAO
- PESSOA
- TEMPO

Formato esperado:
[
  { "text": "Supremo Tribunal", "label": "ORGANIZACAO"},
  { "text": "Brasília", "label": "LOCAL"}
]

"""
    for ex in few_shot_examples:
        prompt += f"Texto: {truncate_text(ex['text'])}\nEntidades: {json.dumps(ex['entities'], ensure_ascii=False)}\n\n"
    prompt += f"Texto: {truncate_text(test_text)}\nEntidades:"
    return prompt

# Limpa o arquivo de relatório anterior
open("relatorio_folds.txt", "w").close()

all_reports = []
labels = get_all_labels()
mlb = MultiLabelBinarizer(classes=labels)

for i in range(10):
    print(f"\nFold {i}")
    test_file = division_files[i]
    train_files = [f for j, f in enumerate(division_files) if j != i]

    test_data = load_data(os.path.join(base_dir, test_file))
    train_data = []
    for tf in train_files:
        train_data.extend(load_data(os.path.join(base_dir, tf)))

    few_shot = select_minimal_few_shot(train_data)
    print(f"Exemplos few-shot usados: {len(few_shot)}")

    print("Exemplos few-shot usados para este fold:")
    for ex in few_shot:
        print(f"Texto: {ex['text']}")
        print(f"Entidades: {json.dumps(ex['entities'], ensure_ascii=False)}")
        print("-" * 50)

    print(f"Total de exemplos no fold de teste: {len(test_data)}")
    print("-" * 50)

    y_true_list, y_pred_list = [], []

    for item in tqdm(test_data):
        start_time = time.time()
        prompt = build_prompt(few_shot, item['text'])

        response = model(prompt, max_tokens=512, stop=["Texto:"])
        exec_time = time.time() - start_time
        print(f"Tempo de execução para exemplo: {exec_time:.4f} segundos")

        try:
            output_text = response["choices"][0]["text"].strip()
            if not output_text.startswith("["):
                output_text = output_text[output_text.find("["):]
            pred_entities = json.loads(output_text)
            if not isinstance(pred_entities, list):
                pred_entities = []
        except Exception:
            pred_entities = []

        y_true_labels = [ent['label'] for ent in item['entities'] if ent['label'] in labels]
        y_pred_labels = [ent['label'] for ent in pred_entities if isinstance(ent, dict) and 'label' in ent and ent['label'] in labels]

        y_true_list.append(y_true_labels)
        y_pred_list.append(y_pred_labels)

    y_true_bin = mlb.fit_transform(y_true_list)
    y_pred_bin = mlb.transform(y_pred_list)

    report = classification_report(y_true_bin, y_pred_bin, target_names=labels, output_dict=True, zero_division=0)
    all_reports.append(report)

    # ===== Salva relatório do fold no arquivo =====
    report_lines = []

    report_lines.append(f"\n====== FOLD {i} ======")
    report_lines.append("Relatório por label:")

    for label in labels:
        report_lines.append(f"- {label}: "
                            f"precision={report[label]['precision']:.3f}, "
                            f"recall={report[label]['recall']:.3f}, "
                            f"f1-score={report[label]['f1-score']:.3f}")

    report_lines.append(f"\nMacro Avg: "
                        f"precision={report['macro avg']['precision']:.3f}, "
                        f"recall={report['macro avg']['recall']:.3f}, "
                        f"f1-score={report['macro avg']['f1-score']:.3f}")
    report_lines.append(f"Micro Avg: "
                        f"precision={report['micro avg']['precision']:.3f}, "
                        f"recall={report['micro avg']['recall']:.3f}, "
                        f"f1-score={report['micro avg']['f1-score']:.3f}")
    report_lines.append("=" * 50)

    with open("relatorio_folds.txt", "a", encoding="utf-8") as f:
        f.write("\n".join(report_lines) + "\n")

# ===== Relatório Final =====
final_report = {}
label_averages = {label: {"precision": 0, "recall": 0, "f1": 0} for label in labels}

for label in labels:
    precision = sum(r[label]['precision'] for r in all_reports if label in r) / len(all_reports)
    recall = sum(r[label]['recall'] for r in all_reports if label in r) / len(all_reports)
    f1 = sum(r[label]['f1-score'] for r in all_reports if label in r) / len(all_reports)
    label_averages[label] = {"precision": precision, "recall": recall, "f1": f1}

macro_precision = sum(r["macro avg"]["precision"] for r in all_reports) / len(all_reports)
macro_recall = sum(r["macro avg"]["recall"] for r in all_reports) / len(all_reports)
macro_f1 = sum(r["macro avg"]["f1-score"] for r in all_reports) / len(all_reports)

micro_precision = sum(r["micro avg"]["precision"] for r in all_reports) / len(all_reports)
micro_recall = sum(r["micro avg"]["recall"] for r in all_reports) / len(all_reports)
micro_f1 = sum(r["micro avg"]["f1-score"] for r in all_reports) / len(all_reports)

# Salva resumo final no arquivo
final_lines = []
final_lines.append("\n===== MÉDIA DOS 10 FOLDS =====")
for label in labels:
    final_lines.append(f"- {label}: "
                       f"precision={label_averages[label]['precision']:.3f}, "
                       f"recall={label_averages[label]['recall']:.3f}, "
                       f"f1-score={label_averages[label]['f1']:.3f}")

final_lines.append("\nMédias finais:")
final_lines.append(f"Macro Avg: precision={macro_precision:.3f}, recall={macro_recall:.3f}, f1-score={macro_f1:.3f}")
final_lines.append(f"Micro Avg: precision={micro_precision:.3f}, recall={micro_recall:.3f}, f1-score={micro_f1:.3f}")

with open("relatorio_folds.txt", "a", encoding="utf-8") as f:
    f.write("\n".join(final_lines) + "\n")




llama_model_loader: loaded meta data with 30 key-value pairs and 579 tensors from /shared/ModelsGGUF/hugging-quants/DeepSeek-R1-Distill-Qwen-14B-GGUF/DeepSeek-R1-Distill-Qwen-14B-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = DeepSeek R1 Distill Qwen 14B
llama_model_loader: - kv   3:                           general.basename str              = DeepSeek-R1-Distill-Qwen
llama_model_loader: - kv   4:                         general.size_label str              = 14B
llama_model_loader: - kv   5:                          qwen2.block_count u32              = 48
llama_model_loader: - kv   6:                       qwen2.c

Usando dispositivo: cuda
GPU disponível: NVIDIA GeForce RTX 3090


load: token to piece cache size = 0.9310 MB
print_info: arch             = qwen2
print_info: vocab_only       = 0
print_info: n_ctx_train      = 131072
print_info: n_embd           = 5120
print_info: n_layer          = 48
print_info: n_head           = 40
print_info: n_head_kv        = 8
print_info: n_rot            = 128
print_info: n_swa            = 0
print_info: n_embd_head_k    = 128
print_info: n_embd_head_v    = 128
print_info: n_gqa            = 5
print_info: n_embd_k_gqa     = 1024
print_info: n_embd_v_gqa     = 1024
print_info: f_norm_eps       = 0.0e+00
print_info: f_norm_rms_eps   = 1.0e-05
print_info: f_clamp_kqv      = 0.0e+00
print_info: f_max_alibi_bias = 0.0e+00
print_info: f_logit_scale    = 0.0e+00
print_info: f_attn_scale     = 0.0e+00
print_info: n_ff             = 13824
print_info: n_expert         = 0
print_info: n_expert_used    = 0
print_info: causal attn      = 1
print_info: pooling type     = 0
print_info: rope type        = 2
print_info: rope scaling     = l


Fold 0
Exemplos few-shot usados: 6
Exemplos few-shot usados para este fold:
Texto: Rio Branco , 27 de março de 2018 .
Entidades: [{"text": "Rio Branco", "label": "LOCAL"}, {"text": "27 de março de 2018", "label": "TEMPO"}]
--------------------------------------------------
Texto: Regina Ferrari Apelante : Acrediesel Comércio de Veículos Ltda Advogado : Thales Rocha Bordignon ( OAB : 2160/AC ) Advogada : Vanessa Fantin Mazoca de Almeida Prado ( OAB : 3956/AC ) Advogado : Mayara Cristine Bandeira de Lima ( OAB : 3580/AC ) Advogado : Gilliard Nobre Rocha ( OAB : 2833/AC ) Apelado : Auto Viação Floresta Cidade de Rio Branco Ltda Advogada : Márcia de Souza Nepomuceno ( OAB : 4181/RO ) Advogado : KLEIR SILVA CARVALHO ( OAB : 3432/AC ) Assunto : Direito Civil APELAÇÃO CÍVEL .
Entidades: [{"text": "Regina Ferrari", "label": "PESSOA"}, {"text": "Acrediesel Comércio de Veículos Ltda", "label": "ORGANIZACAO"}, {"text": "Thales Rocha Bordignon", "label": "PESSOA"}, {"text": "OAB", "label": "ORGAN

llama_perf_context_print:        load time =  164250.05 ms0/775 [00:00<?, ?it/s]
llama_perf_context_print: prompt eval time =  164249.62 ms /  1484 tokens (  110.68 ms per token,     9.04 tokens per second)
llama_perf_context_print:        eval time =   32774.31 ms /   106 runs   (  309.19 ms per token,     3.23 tokens per second)
llama_perf_context_print:       total time =  197126.50 ms /  1590 tokens
Llama.generate: 1418 prefix-match hit, remaining 33 prompt tokens to eval13s/it]


...

Tempo de execução para exemplo: 16.9962 segundos


llama_perf_context_print:        load time =  164250.05 ms
llama_perf_context_print: prompt eval time =    9592.18 ms /    17 tokens (  564.25 ms per token,     1.77 tokens per second)
llama_perf_context_print:        eval time =    8959.22 ms /    30 runs   (  298.64 ms per token,     3.35 tokens per second)
llama_perf_context_print:       total time =   18580.78 ms /    47 tokens
Llama.generate: 1418 prefix-match hit, remaining 14 prompt tokens to eval22s/it]


Tempo de execução para exemplo: 18.5849 segundos


llama_perf_context_print:        load time =  164250.05 ms
llama_perf_context_print: prompt eval time =   15253.31 ms /    14 tokens ( 1089.52 ms per token,     0.92 tokens per second)
llama_perf_context_print:        eval time =    4731.65 ms /    16 runs   (  295.73 ms per token,     3.38 tokens per second)
llama_perf_context_print:       total time =   20000.09 ms /    30 tokens
Llama.generate: 1418 prefix-match hit, remaining 23 prompt tokens to eval71s/it]


Tempo de execução para exemplo: 20.0044 segundos


In [4]:
import torch

# Verifique se a GPU está disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Crie um tensor de exemplo
input_tensor = torch.randn(1000, 1000)  # Um tensor de 1000x1000 com valores aleatórios

# Mova o tensor para a GPU (se disponível)
input_tensor = input_tensor.to(device)

# Verifique se o tensor foi movido para a GPU
print(input_tensor.device)  # Deve mostrar algo como 'cuda:0' se estiver usando a GPU


cuda:0


In [3]:
import torch

# Verifique se a GPU está disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Carregue o modelo
model = seu_modelo()  # Substitua por seu modelo real

# Mova o modelo para a GPU
model.to(device)

# Se necessário, mova os dados (tensores) para a GPU também
input_tensor = input_tensor.to(device)


NameError: name 'seu_modelo' is not defined

In [ ]:
label_list = ner_dataset["train"].features[f"ner_tags"].feature.names
label_list

In [ ]:
# BERT pretrained model
model_id = 'neuralmind/bert-base-portuguese-cased'

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
import transformers

assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
tokenizer("Olá, isto é uma sentença!")

In [ ]:
example = ner_dataset["train"][2]
print(example["tokens"])

In [ ]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True, truncation=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

In [ ]:
len(example[f"ner_tags"]), len(tokenized_input["input_ids"])

In [ ]:
print(tokenized_input.word_ids())

In [ ]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"ner_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

In [ ]:
label_all_tokens = True

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_datasets = ner_dataset.map(tokenize_and_align_labels, batched=True, load_from_cache_file=False)

In [ ]:
tokenized_datasets["train"]["labels"][2]

In [ ]:
from transformers import TFAutoModelForTokenClassification

model = TFAutoModelForTokenClassification.from_pretrained(
    model_id, num_labels=len(label_list), from_pt=True
)

In [ ]:
batch_size = 8

In [ ]:
from transformers import create_optimizer

num_train_epochs = 3
num_train_steps = (len(tokenized_datasets["train"]) // batch_size) * num_train_epochs
optimizer, lr_schedule = create_optimizer(
    init_lr=2e-5,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
    num_warmup_steps=0,
)

In [ ]:
import tensorflow as tf

model.compile(optimizer=optimizer)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer, return_tensors="tf")

In [ ]:
train_set = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)
validation_set = tokenized_datasets["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)
test_set = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

In [ ]:
import numpy as np
from datasets import load_metric
from transformers.keras_callbacks import KerasMetricCallback

metric = load_metric("seqeval")
labels = [label_list[i] for i in example[f"ner_tags"]]
metric.compute(predictions=[labels], references=[labels])


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


metric_callback = KerasMetricCallback(
    metric_fn=compute_metrics, eval_dataset=validation_set
)

In [ ]:
from transformers.keras_callbacks import PushToHubCallback
# from tensorflow.keras.callbacks import TensorBoard

# model_name = model_checkpoint.split("/")[-1]
# push_to_hub_model_id = f"{model_name}-finetuned-{task}"

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./tc_model_save/logs")

# push_to_hub_callback = PushToHubCallback(
#     output_dir="./tc_model_save",
#     tokenizer=tokenizer,
#     hub_model_id=push_to_hub_model_id,
# )

callbacks = [metric_callback, tensorboard_callback]

model.fit(
    
    train_set,
    validation_data=validation_set,
    epochs=num_train_epochs,
    callbacks=callbacks,
)

In [ ]:
import os

db_path = "optuna_study.db"

if os.path.exists(db_path):
    os.remove(db_path)
    print("Banco de dados deletado com sucesso.")
else:
    print("O banco de dados não existe.")





In [ ]:
import sqlite3

conn = sqlite3.connect("optuna_study.db")
cursor = conn.cursor()

cursor.execute("SELECT * FROM trials")
print(cursor.fetchall())

conn.close()




In [ ]:
model.predict(validation_set)

In [ ]:
model.save_pretrained("./modelNER")

In [ ]:
model.config.id2label.values()

In [ ]:
from datasets import load_metric
import numpy as np


metric = load_metric("seqeval")


def evaluate(model, dataset, ner_labels):
  all_predictions = []
  all_labels = []
  for batch in dataset:
      logits = model.predict(batch)["logits"]
      labels = batch["labels"]
      predictions = np.argmax(logits, axis=-1)
      for prediction, label in zip(predictions, labels):
          for predicted_idx, label_idx in zip(prediction, label):
              if label_idx == -100:
                #   print(label)
                  continue
              all_predictions.append(ner_labels[predicted_idx])
              all_labels.append(ner_labels[label_idx])
              #print('\npredicted=',ner_labels[predicted_idx], '\nlabel=',ner_labels[label_idx])
  #print("all_predictions=",[all_predictions],'\nall_labels=',[all_labels])
  return metric.compute(predictions=[all_predictions], references=[all_labels])

#results = evaluate(model, tf_eval_dataset, ner_labels=list(model.config.id2label.values()))
results = evaluate(model, test_set, ner_labels=list(model.config.id2label.values()))
results